## 시설물 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 목록/selectIoFmMngList.csv'

In [17]:
# 전체 시설물 목록 수 (7913)
global g_totalCount

In [2]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmMngList'

In [4]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
REQPARAM

['serviceKey',
 'type',
 'pageNo',
 'numOfRows',
 'sortField',
 'sortOrder',
 'searchFctNm',
 'searchFctCfCd',
 'searchFctKdCd',
 'searchRutCd',
 'searchCwkNm',
 'searchOfcCd']

In [5]:
JSONKEY = "items"
DUMMY = 0

## 시설물 목록 전용 스크랩 함수

In [18]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            
            response = requests.get(inUrl+queryParams)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata == "NODATA_ERROR"):
                        print("{} page is empty".format(i))
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    g_totalCount = int(response.json()["response"]["body"]["totalCount"])
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [19]:
pageNo = 1
numOfRows = 1000
PAGEYN=0

In [20]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

while True:
    BASEPARAM={"serviceKey":APIKEY,"pageNo": pageNo,"numOfRows": numOfRows, "type":"json"}
    resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
    if resultDf.empty: # 정상 데이터가 없는 경우
        break
    else:
        resultDfMerged = resultDfMerged.append(resultDf) 
        global g_totalCount
        g_totalCount -= numOfRows
        print("totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
        if(g_totalCount <= 0):
            break
        pageNo += 1
        

        

1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
>totalCount 7913
0 no pageNo
totalCount :  6913  numOfRows :  1000
1 page scraping start
1 page is empty


In [21]:
resultDfMerged.shape

(7913, 15)

In [22]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

시설물 목록 save compled


In [23]:
resultDfMerged.shape

(7913, 15)